In [1]:
import numpy.random
import torch
from torch import nn
import numpy as np

In [2]:
num_examples, num_inputs = 2000, 4
features = torch.randn(num_examples, num_inputs)
true_w = torch.tensor([1.1, -2.2, 3.3, -4.4]).view(4, 1)
true_b = torch.tensor([5.5])
labels = torch.matmul(features, true_w) + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()))
features.size(), labels.size()

(torch.Size([2000, 4]), torch.Size([2000, 1]))

In [3]:
import torch.utils.data as Data
dataset = Data.TensorDataset(features, labels)
batch_size = 10
data_iter = Data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [4]:
for X, y in data_iter:
    print(X, y)
    break
net = nn.Sequential(nn.Linear(num_inputs, 1))
net

tensor([[ 0.1043, -0.2737, -1.2184,  0.0766],
        [ 0.5457, -0.6421,  2.5052,  0.7603],
        [ 0.7044,  0.9190,  0.7113, -0.0784],
        [ 0.4407,  1.6817,  0.0598,  0.5525],
        [ 1.5750, -1.2688, -0.0160, -0.1599],
        [ 0.5125,  1.2878,  0.5106,  0.9108],
        [ 0.8015, -0.4470, -1.1270, -0.6111],
        [ 1.5454,  0.0655, -0.6805, -0.8853],
        [-2.9616,  0.6592, -0.2989,  0.2091],
        [ 0.7317,  1.2542, -0.0794,  0.4616]]) tensor([[ 1.8642],
        [12.4572],
        [ 6.9527],
        [ 0.0670],
        [10.6835],
        [ 0.8986],
        [ 6.3506],
        [ 8.6983],
        [-1.1201],
        [ 1.2519]])


Sequential(
  (0): Linear(in_features=4, out_features=1, bias=True)
)

In [5]:
loss = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.02)

In [6]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.2507, -0.3688,  0.3009,  0.3950]], requires_grad=True)
Parameter containing:
tensor([-0.4844], requires_grad=True)


In [7]:
from torch.nn import init
init.constant_(net[0].bias.data, val=0)
init.normal_(net[0].weight, mean=0, std=0.01)

Parameter containing:
tensor([[-0.0099,  0.0068,  0.0030, -0.0005]], requires_grad=True)

In [8]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0099,  0.0068,  0.0030, -0.0005]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


In [9]:
x_test = torch.tensor([0, 0, 0, 1], dtype=torch.float32).view(1, 4);
net(x_test)

tensor([[-0.0005]], grad_fn=<AddmmBackward0>)

In [10]:
num_epochs = 3
for epoch in range (1, num_epochs + 1):
    for X, y in data_iter:
        y_hat = net(X)
        l = loss(y_hat.view(-1, 1), y.view(-1, 1))
        net.zero_grad()
        l.backward()
        optimizer.step()
    print("epoch %d, loss=%.5f" % (epoch, l.item()))

epoch 1, loss=0.00012
epoch 2, loss=0.00006
epoch 3, loss=0.00012


In [11]:
true_w, true_b

(tensor([[ 1.1000],
         [-2.2000],
         [ 3.3000],
         [-4.4000]]),
 tensor([5.5000]))

In [12]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 1.0999, -2.2002,  3.3001, -4.3993]], requires_grad=True)
Parameter containing:
tensor([5.4999], requires_grad=True)
